In [1]:
%reload_ext autoreload
%autoreload 2

# Use aligner_v7 kernel

import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    "/fsx_0/shared/conda/aligner_20241030/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
device = "cuda:0"

In [3]:
import torch
from llm_mm_aligner.lib.encoders.metaclip_native_resolution_inference import MetaCLIP_NAVIT as MetaCLIP_NAVIT_INF

from llm_mm_aligner.lib.encoders.metaclip_navit import MetaCLIP_NAVIT
from PIL import Image
from torch import nn
from torchvision import transforms

# Test load state_dict

In [4]:
# test loading
MODEL_H = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
MODEL_G = "/fsx_0/user/marcomonteiro/experiments/G/VEv0p2_G14_aws_final_256x8x98_next/checkpoints/epoch_450.pt"

sd_h = torch.load(MODEL_H, map_location="cpu")
sd_g = torch.load(MODEL_G, map_location="cpu")

In [20]:
_sd.keys()

dict_keys(['epoch', 'name', 'state_dict', 'optimizer'])

In [ ]:

_sd = sd_g
positional_embedding = None 

# if the checkpoint contains both visual and text modules, we only take the visual module
if 'state_dict' in _sd:
    print("yes")
    visual_module_prefix = "module.visual."
    visual_sd = {}
    for k, v in _sd["state_dict"].items():
        if k.startswith(visual_module_prefix):
            visual_sd[k.replace(visual_module_prefix, "")] = v
            
    if visual_sd:
        _sd = visual_sd
    else:
        _sd = _sd['state_dict']

# dropping positional_embedding and other unwanted layers
if ("positional_embedding" in _sd) and (positional_embedding is None):
    # Special case for VEv0.1 and VEv0.2 ckpt. The ckpt has positional embedding, but the model does not.
    _sd.pop("positional_embedding")
    print(
        "MetaCLIP_NAVIT dropping positional_embedding from checkpoint"
    )

drop_layers = [
    "mim_token",
    "proj",
    "attn_pool.probe",
    "attn_pool.attn.in_proj_weight",
    "attn_pool.attn.in_proj_bias",
    "attn_pool.attn.out_proj.weight",
    "attn_pool.attn.out_proj.bias",
    "attn_pool.layernorm.weight",
    "attn_pool.layernorm.bias",
    "attn_pool.mlp.c_fc.weight",
    "attn_pool.mlp.c_fc.bias",
    "attn_pool.mlp.c_proj.weight",
    "attn_pool.mlp.c_proj.bias",
]

loaded_layers = list(_sd.keys())
for k in loaded_layers:
    if k in drop_layers or k.endswith("attn.rope.freqs"):
        print(f"MetaCLIP_NAVIT dropping {k} from checkpoint")
        _sd.pop(k)

yes
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping mim_token from checkpoint
MetaCLIP_NAVIT dropping proj from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.0.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.1.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.2.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.3.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.4.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.5.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.6.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.7.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.8.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.9.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resb

# Init model

In [17]:
MODEL_CHECKPOINT = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
# MODEL_CHECKPOINT = "/fsx_0/user/marcomonteiro/experiments/G/VEv0p2_G14_aws_final_256x8x98_next/checkpoints/epoch_450.pt"
TEST_IMAGE = "/fsx_0/dataset01/coco/val2014/COCO_val2014_000000485307.jpg"

# ViT-H
metaclip_config = {
    "ckpt_path": MODEL_CHECKPOINT,
    "image_size": 336,  # image_size can also be 448, since the position embedding is rope_2d
    "patch_size": 14,
    "width": 1408,
    "mlp_ratio": 4.0,
    "layers": 34,
    "heads": 16,
    "load_ckpt": True,
    "relative_pos_embed_type": "rope_2d",
}

# ViT_G
# metaclip_config = {
#     "ckpt_path": MODEL_CHECKPOINT,
#     "image_size": 392,  # image_size can also be 448, since the position embedding is rope_2d
#     "patch_size": 14,
#     "width": 1536,
#     "mlp_ratio": 5.833333334,
#     "layers": 50,
#     "heads": 16,
#     "load_ckpt": True,
#     "relative_pos_embed_type": "rope_2d",
#     "embed_cls_token": False
# }

CLIP_MEAN = (0.4814546, 0.4578275, 0.40821073)
CLIP_STD = (0.2686295, 0.2613025, 0.2757711)

preprocess = transforms.Compose(
    [
        transforms.Resize(metaclip_config["image_size"]),
        transforms.CenterCrop(metaclip_config["image_size"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4814546, 0.4578275, 0.40821073),
            std=(0.2686295, 0.2613025, 0.2757711),
        ),
    ]
)

In [14]:
image = Image.open(TEST_IMAGE).convert("RGB")
preprocessed_image = preprocess(image).unsqueeze(0).to(device)

In [18]:
model = MetaCLIP_NAVIT(**metaclip_config).to(device)
model.eval()

MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT Visual Pretrained Model Missing keys: []
MetaCLIP_NAVIT Visual Pretrained Model Unexpected keys: []


MetaCLIP_NAVIT(
  (conv1): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-33): 34 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionXformer4x(
          (out_proj): Linear(in_features=1408, out_features=1408, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1408, out_features=5632, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=5632, out_features=1408, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
model2 = MetaCLIP_NAVIT_INF(**metaclip_config).to(device)
model2.eval()

In [22]:
with torch.no_grad():
    output = model(preprocessed_image)
output.shape

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_navit.py:337: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


torch.Size([1, 576, 1408])

In [ ]:
with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    output2 = model2(preprocessed_image)
output2.shape

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_native_resolution_inference.py:899: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


torch.Size([1, 576, 1408])

In [26]:
output

tensor([[[ 2.4852e+00,  8.1081e-01,  6.0595e-01,  ..., -1.9258e+00,
          -1.2234e-01,  4.2518e+00],
         [ 3.5210e-01,  3.2244e+00, -6.3592e+00,  ..., -3.6949e-01,
          -1.1734e+00,  3.9925e+00],
         [ 2.7544e+00,  7.4645e-01, -4.8141e+00,  ..., -1.6364e+00,
          -5.5741e+00,  6.2823e-03],
         ...,
         [-1.8769e+00,  2.5796e+00,  2.5895e+00,  ...,  5.2259e+00,
          -6.4294e-01,  4.7235e+00],
         [ 7.9809e-01,  1.3053e+00, -1.4412e+00,  ..., -1.3024e+00,
          -6.2229e+00, -9.8805e-01],
         [-2.4709e+00, -3.5142e-01,  5.6059e-02,  ..., -3.4691e+00,
          -3.0556e+00,  3.3854e+00]]], device='cuda:0')

In [27]:
output2

tensor([[[ 1.1328e+00, -5.7031e-01,  3.9844e-01,  ..., -1.2500e+00,
          -3.9062e-03,  4.9375e+00],
         [ 3.7109e-01,  3.8594e+00, -6.5312e+00,  ..., -3.2031e-01,
          -1.1250e+00,  3.8438e+00],
         [ 2.7188e+00,  9.1797e-01, -4.6562e+00,  ..., -1.7344e+00,
          -5.4062e+00,  9.3750e-02],
         ...,
         [-1.0938e+00,  2.2188e+00,  1.5312e+00,  ...,  1.2344e+00,
           1.4844e-01,  4.0000e+00],
         [-2.1875e+00,  1.4844e+00,  1.8594e+00,  ...,  2.0781e+00,
          -1.0781e+00,  3.0625e+00],
         [ 1.2344e+00,  3.1406e+00,  4.5312e-01,  ..., -1.3281e+00,
          -4.6875e+00,  1.5312e+00]]], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<ViewBackward0>)

In [24]:
difference = output2 - output


# Calculate the norm of the difference
norm = torch.norm(difference)
norm

tensor(1024.1246, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)

In [35]:
with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    output1 = model(preprocessed_image)

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_navit.py:337: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


In [37]:
print("diff 2:", torch.norm(output2 - output1))
print("diff 3:", torch.norm(output3 - output1))

diff 2: tensor(0., device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<LinalgVectorNormBackward0>)
diff 3: tensor(1024.1246, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)


In [32]:
with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    output2 = model2(preprocessed_image)
print("diff 2:", torch.norm(output2 - output))
 
with torch.no_grad():
    output3 = model2(preprocessed_image)
print("diff 3:", torch.norm(output3 - output))

diff 2: tensor(1024.1246, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
diff 3: tensor(0., device='cuda:0')


/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_native_resolution_inference.py:899: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


In [30]:
torch.norm(output3 - output), torch.norm(output2 - output)

(tensor(0., device='cuda:0'),
 tensor(1024.1246, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>))